In [ ]:
import os
from tqdm import tqdm

In [ ]:
from avapi.carla import CarlaScenesManager


cpath = os.path.join("/data/shared/CARLA/multi-agent-v1/")
# cpath = os.path.join("/data/shared/CARLA/multi-agent-intersection/")
# cpath = "../examples/sim_results"
CSM = CarlaScenesManager(cpath)
print(CSM.scenes)
CDM = CSM.get_scene_dataset_by_name(CSM.splits_scenes["val"][0])
print(f"{len(CDM)} frames")

## Test Different Perception/Tracking Approaches

In [ ]:
from avstack.geometry import Sphere, Circle

from avstack.modules.perception.object2dfv import MMDetObjectDetector2D
from avstack.modules.perception.object3d import Passthrough3DObjectDetector, MMDetObjectDetector3D
from avstack.modules.tracking.tracker3d import BasicBoxTracker3D
from avstack.modules.tracking.multisensor import MeasurementBasedMultiTracker

from avstack.datastructs import DataContainer

# init models
agents = list(range(10))
percep = Passthrough3DObjectDetector()
percep_veh = MMDetObjectDetector3D(model="pointpillars", dataset="carla-vehicle")
percep_inf = MMDetObjectDetector3D(model="pointpillars", dataset="carla-infrastructure")
trackers = {agent: BasicBoxTracker3D() for agent in agents}
trackers["central"] = MeasurementBasedMultiTracker(tracker=BasicBoxTracker3D())

# run loop
dets = {}
tracks = {}
imgs_all = {agent: [] for agent in agents}
pcs_all = {agent: [] for agent in agents}
dets_all = {agent: [] for agent in agents}
tracks_all = {agent: [] for agent in agents}
tracks_all["central"] = []
agent_0_frames = CDM.get_frames(sensor="lidar-0", agent=0)[1:-1]
for frame in tqdm(agent_0_frames):
    # run perception and individual trackers
    found_data = False
    fovs = {}
    platforms = {}
    for agent in agents:
        ###############################################
        # GET DATA
        ###############################################
        lidar_sensor = "lidar-0"
        camera_sensor = "camera-0"
        img = CDM.get_image(frame=frame, sensor=camera_sensor, agent=agent)
        pc = CDM.get_lidar(frame=frame, sensor=lidar_sensor, agent=agent)
        imgs_all[agent].append(img)
        pcs_all[agent].append(pc)
        objs = CDM.get_objects(frame=frame, sensor=lidar_sensor, agent=agent)
        calib = CDM.get_calibration(frame=frame, sensor=lidar_sensor, agent=agent)
        fovs[agent] = Sphere(radius=100)
        platforms[agent] = calib.reference

        ###############################################
        # DISTRIBUTED PERCEPTION
        ###############################################
        found_data = True
        dets[agent] = percep(objs)
        dets_all[agent].append(dets[agent])

        ###############################################
        # DISTRIBUTED TRACKING USING DISTRIBUTED PERCEP
        ###############################################
        tracks[agent] = trackers[agent](dets[agent], platform=calib.reference)
        if not isinstance(tracks[agent], DataContainer):
            raise
        tracks_all[agent].append([track.box3d.copy() for track in tracks[agent]])

    ###############################################
    # CENTRALIZED PERCEPTION
    ###############################################
    ######
    # TODO: QINGZHAO TO COMPLETE
    ######
        
    ###############################################
    # CENTRALIZED TRACKING USING DISTRIBUTED PERCEP
    ###############################################
    # run central tracker on all detections
    if found_data:
        tracks["central"] = trackers["central"](
            detections=dets,
            fovs=fovs,
            platforms=platforms,
        )
        tracks_all["central"].append(tracks["central"])




#### Verify that the union of the two is similar to the central

In [ ]:
from avapi.visualize.snapshot import show_lidar_bev_with_boxes, show_boxes_bev

# extent = [[-70, 70], [-70, 70], [-100, 100]]
extent = [[-50, 200], [-200, 50], [-100, 100]]

show_boxes_bev(boxes=tracks["central"])
show_lidar_bev_with_boxes(pc=pc, boxes=tracks["central"], extent=extent)

In [ ]:
from avapi.visualize.movie import make_movie


vid_folder = "videos"
os.makedirs(vid_folder, exist_ok=True)



def make_agent_movies(imgs, pcs, dets, tracks, agent, extent=None):

    ###############################################
    # CAMERA-BASED VISUALIZATION
    ###############################################

    # perception movie
    make_movie(
        raw_imgs=imgs,
        raw_pcs=pcs,
        boxes=dets,
        name=os.path.join(vid_folder, f"agent-{agent}-perception-img"),
        save=True,
        show_in_notebook=False,
        projection="img",
        extent=extent,
    )

    # tracking movie
    make_movie(
        raw_imgs=imgs,
        raw_pcs=pcs,
        boxes=tracks,
        name=os.path.join(vid_folder, f"agent-{agent}-tracking-img"),
        save=True,
        show_in_notebook=False,
        projection="img",
        extent=extent,
    )

    ###############################################
    # BEV-BASED VISUALIZATION
    ###############################################

    # perception movie
    # make_movie(
    #     raw_imgs=imgs,
    #     raw_pcs=pcs,
    #     boxes=dets,
    #     name=os.path.join(vid_folder, f"agent-{agent}-perception-bev"),
    #     save=True,
    #     show_in_notebook=False,
    #     projection="bev",
    #     extent=extent,
    # )

    # # tracking movie
    # make_movie(
    #     raw_imgs=imgs,
    #     raw_pcs=pcs,
    #     boxes=tracks,
    #     name=os.path.join(vid_folder, f"agent-{agent}-tracking-bev"),
    #     save=True,
    #     show_in_notebook=False,
    #     projection="bev",
    #     extent=extent,
    # )


# extent = [[-70, 70], [-70, 70], [-100, 100]]
# for agent in agents:
#     make_agent_movies(
#         imgs=imgs_all[agent],
#         pcs=pcs_all[agent],
#         dets=dets_all[agent],
#         tracks=tracks_all[agent],
#         agent=agent,
#         extent=extent,
#     )


In [ ]:
import numpy as np
from avapi.visualize.movie import make_movie
from avstack.geometry import GlobalOrigin3D, PointMatrix3D
from avstack.calibration import LidarCalibration
from avstack.sensors import LidarData


vid_folder = "videos"
os.makedirs(vid_folder, exist_ok=True)



def make_central_movie(pcs_all, tracks, extent=None):
    # aggregate all point clouds in global frame
    calib_global = LidarCalibration(GlobalOrigin3D)
    pc_data = [[] for _ in range(len(list(pcs_all.values())[0]))]
    for agent, pcs_agent in pcs_all.items():
        for frame, pc in enumerate(pcs_agent):
            pc = pc.project(calib_global)
            pc_data[frame].append(pc.data.x)
    pcs = []
    for i in range(len(pc_data)):
        pc_data[i] = PointMatrix3D(x=np.concatenate(pc_data[i], axis=0), calibration=calib_global)
        pcs.append(
            LidarData(
                data=pc_data[i],
                timestamp=0,
                frame=i,
                source_name="lidar",
                source_ID=0,
                calibration=calib_global,
            )
        )

    # tracking movie
    print("making movie")
    make_movie(
        raw_imgs=[[]] * len(pcs),
        raw_pcs=pcs,
        boxes=tracks,
        name=os.path.join(vid_folder, f"central-tracking-bev"),
        save=True,
        show_in_notebook=False,
        projection="bev",
        extent=extent,
    )


make_central_movie(pcs_all, tracks_all["central"])
